In [25]:
import os
import pandas as pd
import plotly.graph_objects as go

### Create a fig folder to store the analysis figures

In [26]:
if not os.path.exists("fig"):
    os.makedirs("fig")

## Graph Requirements

- Queries per second (QPS) on the x-axis (the x-axis should range from 0 to 80K). (note: the actual achieved QPS, not the target QPS)
- 95th percentile latency on the y-axis (the y-axis should range from 0 to 6 ms).
- Label your axes.
- 7 lines, one for each configuration. Add a legend.
- State how many runs you averaged across and include error bars at each point in both
dimensions

### There are 7 configurations
- without interference &rarr; some-memcached_run.txt
- cpu &rarr; ibench-cpu_run.txt
- l1d &rarr; ibench-l1d_run.txt
- l1i &rarr; ibench-l1i_run.txt
- l2 &rarr; ibench-l2_run.txt
- llc &rarr; ibench-llc_run.txt
- membw &rarr; ibench-membw_run-1.txt

In [27]:
categories = ["without_intereference", "cpu", "l1d", "l1i", "l2", "llc", "membw"]
benchmark_file = {}

for category in categories:
    benchmark_file[category] = []
    if category == "without_intereference":
        benchmark_file[category].append(f"some-memcached")
    else:
        benchmark_file[category].append(f"ibench-{category}")


In [28]:
dir_path = "benchmarks"
file_list = os.listdir(dir_path)

df_list = []
for filename in file_list:
    df = pd.read_csv(f"{dir_path}/{filename}", nrows=16, header=1, sep="\\s+")
    df["filename"] = filename.split("_")[0]
    df_list.append(df)

df = pd.concat(df_list, ignore_index=True)
df

,#type,avg,std,min,p5,p10,p50,p67,p75,p80,p85,p90,p95,p99,p999,p9999,QPS,target,filename
0,read,208.0,92.0,80.4,139.7,151.6,203.3,220.4,230.2,236.5,246.8,258.4,275.3,318.9,1506.5,3821.8,4968.8,5000,some-memcached
1,read,234.9,78.5,86.8,139.1,158.2,232.6,254.7,267.5,275.5,287.5,301.1,323.4,373.6,1070.2,2470.9,9993.2,10000,some-memcached
2,read,244.5,121.5,74.4,129.6,146.9,242.3,267.9,282.1,292.3,305.1,320.3,346.1,411.1,1900.4,4932.4,15035.3,15000,some-memcached
3,read,252.3,95.5,74.4,126.3,142.5,254.4,284.0,299.5,311.7,325.3,344.0,372.8,436.8,677.2,2421.4,20078.8,20000,some-memcached
4,read,264.6,104.5,74.4,124.6,141.2,265.5,301.4,320.6,335.9,352.9,373.8,405.7,480.7,819.5,2668.6,24981.3,25000,some-memcached
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,read,1313.9,718.3,74.4,879.3,948.1,1160.8,1285.8,1422.0,1497.7,1574.3,1700.4,2126.9,3229.9,8091.0,10044.3,51431.8,60000,ibench-l1d
332,read,1370.5,753.6,86.8,927.9,980.4,1190.6,1372.8,1477.7,1538.1,1597.2,1712.4,2176.2,5044.2,8117.2,10385.7,50058.0,65000,ibench-l1d
333,read,1271.9,672.9,93.8,900.6,954.6,1131.2,1197.9,1258.6,1327.5,1470.8,1655.5,2123.6,3069.1,8070.7,9997.7,53543.0,70000,ibench-l1d
334,read,1330.6,712.3,160.7,925.6,976.6,1167.5,1269.9,1393.3,1474.9,1559.9,1687.7,2154.6,3328.6,8083.3,9838.7,51453.8,75000,ibench-l1d


In [29]:
agg_df = df.groupby(['filename', 'target']).agg(
    p95_mean=('p95', 'mean'),
    p95_std=('p95', 'std'),
    QPS_mean=('QPS', 'mean'),
    QPS_std=('QPS', 'std'),
).reset_index()

print(agg_df.head())

     filename  target     p95_mean    p95_std      QPS_mean    QPS_std
0  ibench-cpu    5000  1042.300000  61.850061   4991.133333  13.900120
1  ibench-cpu   10000  1340.300000   4.757100  10010.133333  38.814602
2  ibench-cpu   15000  1352.933333   4.481443  14972.600000  63.208544
3  ibench-cpu   20000  1380.300000  11.246777  19963.533333  82.404207
4  ibench-cpu   25000  1417.133333  11.850035  25021.566667  33.338616


In [30]:
fig = go.Figure()

for fname in agg_df["filename"].unique():
    sub_df = agg_df[agg_df["filename"] == fname]

    fig.add_trace(go.Scatter(
        x = sub_df["QPS_mean"],
        y = sub_df["p95_mean"]/1000,
        mode = "lines+markers",
        name = fname,
        marker = dict(size = 8),
        line=dict(width=2),
        error_x=dict(
            type="data",
            array=sub_df["QPS_std"],
            thickness=1.5,
            color='gray',
            width=4
        ),
        error_y=dict(
            type="data",
            array=sub_df["p95_std"]/1000,
            thickness=1.5,
            color='gray',
            width=4
        ),
        text=sub_df['target'],
        hovertemplate=(
        "Achieved QPS: %{x}<br>" +
        "95th Latency: %{y} ms<br>" +
        "Target QPS: %{text}<extra></extra>"
        )
    ))

# Layout setting
fig.update_layout(
    title = "Part 1: 95 Percentile Latency vs Achieved QPS",
    xaxis_title = "QPS (actual achieved rather than target)",
    yaxis_title="95 Percentile Latency (ms)",
    xaxis = dict(range=[0, 80000]),
    yaxis = dict(range=[0, 6]),
    legend=dict(
        x=0.99,
        y=0.99,
        xanchor="right",
        yanchor="top",
        bgcolor="rgba(255,255,255,0.7)",
        bordercolor="black",
        borderwidth=1,
        font=dict(size=12)
    )
)

fig.add_annotation(
    text="Averaged across 3 runs",
    xref="paper", yref="paper",
    x=0.99, y=0.01,
    showarrow=False,
    font=dict(size=12),
    align="right"
)

fig.show()

Save the figure

In [31]:
# It seems that I can't store the image locally but the html works
# fig.write_image("fig/95_percentile_latency_achieved_qps.png")
fig.write_html("fig/95_percentile_latency_achieved_qps.html")